This notebook requires pyfabm.
For instructions on how to build and install pyfabm, see https://github.com/fabm-model/fabm/wiki/python

In [ ]:
import os
import glob

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (interactive but deprecated) or "%matplotlib inline" (static plots)
%matplotlib widget
from matplotlib import pyplot
import ipywidgets
import numpy as np
import scipy.integrate

import pyfabm

In [ ]:
# Settings
fabm_yaml = 'fabm/testcases/fabm-nersc-ecosmo.yaml'

bottom_depth = 10
default_environment = {
    'temperature': 15,
    'practical_salinity': 35,
    'surface_downwelling_shortwave_flux': 50,
    'surface_downwelling_photosynthetic_radiative_flux': 25,
    'downwelling_photosynthetic_radiative_flux': 10,
    'density': 1020,
    'mole_fraction_of_carbon_dioxide_in_air': 414.2,
    'wind_speed': 2.0,
    'bottom_depth': bottom_depth,
    'depth': 0.5*bottom_depth,
    'pressure': 0.5*bottom_depth,
    'cell_thickness': bottom_depth
}

In [ ]:
# Initialize the biogeochemical model (this reads fabm.yaml)
model = pyfabm.Model(fabm_yaml)

In [ ]:
# Present configurable environmental conditions
model.cell_thickness = bottom_depth  # cell thickness in m, used by getRates to scale surface and bottom fluxes
labels, inputs, units = [], [], []
for variable in model.dependencies:
    labels.append(ipywidgets.Label('%s:' % variable.long_name))
    inputs.append(ipywidgets.FloatText(value=default_environment.get(variable.name, 0.), layout={'width': '7em'}))
    units.append(ipywidgets.Label('%s' % variable.units))
display(ipywidgets.HBox((
    ipywidgets.VBox([ipywidgets.HTML('<b>Variable</b>')] + labels),
    ipywidgets.VBox([ipywidgets.HTML('<b>Value</b>')] + inputs),
    ipywidgets.VBox([ipywidgets.HTML('<b>Units</b>')] + units)
)))

In [ ]:
# Transfer environmental conditions to model
for variable, widget in zip(model.dependencies, inputs):
    variable.value = widget.value

# Initialize model (this also verifies that all dependencies have been fulfilled)
assert model.start(), 'Model failed to start: %s' % pyfabm.getError()

In [ ]:
# Time integration with scipy - by default variable time step Runge-Kutta 4(5)

# Time derivative
def dy(t, y):
    model.state[:] = y
    return model.getRates(t)

# Time-integrate over 200 days (note: FABM's internal time unit is seconds!)
t = np.arange(0, 365.0, 1) * 86400
result = scipy.integrate.solve_ivp(dy, [t[0], t[-1]], model.state, t_eval=t)

In [ ]:
# Plot results
pyplot.ioff()
fig, ax = pyplot.subplots()
line, = ax.plot(result.t / 86400, result.y[0, :])
ax.grid()
ax.set_xlabel("time (d)")

def update(variable):
    v = model.state_variables[variable]
    line.set_ydata(result.y[variable, :])
    ax.set_ylabel(f"{v.long_name} ({v.units})")
    ax.set_title(v.long_name)
    ax.relim()
    ax.autoscale()
    fig.canvas.draw()
dropdown = ipywidgets.interactive(update, variable=[(variable.long_path, i) for i, variable in enumerate(model.state_variables)])
display(dropdown)
fig.show()

In [ ]:
# Backup: manual Forward Euler integration, fixed dt
dt = 3600.
t = np.arange(365 * 86400 / dt)
y = np.empty((t.size, model.state.size))
y[0, :] = model.state
for i, time in enumerate(t):
    sources = model.getRates(time)
    model.state += sources * dt
    y[i, :] = model.state
from collections import namedtuple
intresulttype = namedtuple('intresult', ['t', 'y'])
result = intresulttype(t=t, y=y.T)